# *Some preprocessing on the device and importing the required libraries.*

In [ ]:
import tensorflow as tf  
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar100.load_data(label_mode='fine')


In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import pickle
import os
import seaborn as sns
import sklearn
import matplotlib.image as mpimg

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

plt.style.use('dark_background')

from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator # Data Augmentation
from keras.layers import BatchNormalization # Has a very good result and in the speed
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten,Dropout
from keras.utils import normalize, to_categorical
from sklearn.model_selection import train_test_split


In [ ]:
# Force memory can only be used by the GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

## Step1: Data acquistion and preprocessing

- ## *1. Labelling all the catergories for the dataset to better visualization after*

In [ ]:
# Cache path for CIFAR-100 dataset
cifar100_path = os.path.expanduser('~/.keras/datasets/cifar-100-python')

# Loading metadata files
def load_cifar100_meta(file_path):
    with open(file_path, 'rb') as f:
        meta = pickle.load(f, encoding='bytes')
    return meta

# Extract fine-grained category names
meta = load_cifar100_meta(os.path.join(cifar100_path, 'meta'))
fine_label_names = [label.decode('utf-8') for label in meta[b'fine_label_names']]

# Print the category name to confirm
print("CIFAR-100 Fine Label Names:")
print(fine_label_names)
print(f"Total number of fine labels: {len(fine_label_names)}")

In [ ]:
label_names = [
    'apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 'bicycle', 'bottle',
    'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 'can', 'castle', 'caterpillar', 'cattle',
    'chair', 'chimpanzee', 'clock', 'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur',
    'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 'house', 'kangaroo', 'keyboard',
    'lamp', 'lawn_mower', 'leopard', 'lion', 'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain',
    'mouse', 'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear', 'pickup_truck', 'pine_tree',
    'plain', 'plate', 'poppy', 'porcupine', 'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket',
    'rose', 'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake', 'spider',
    'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table', 'tank', 'telephone', 'television', 'tiger', 'tractor',
    'train', 'trout', 'tulip', 'turtle', 'wardrobe', 'whale', 'willow_tree', 'wolf', 'woman', 'worm'
]
len(label_names)

- ## *2. Using One-Hot Encoding on y_train and y_test*

In [ ]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

print('The y_train shape has been changed to:',y_train.shape)
print('The y_train shape has been changed to:',y_test.shape)

# The integer labels are converted into a format suitable for the multiclassification task,
# but care needs to be taken with the continuity of the labels and the choice of the loss function.


*The integer labels are converted into a format suitable for the multiclassification task, but care needs to be taken with the continuity of the labels and the choice of the loss function. Then we will choose the softmax as output activations functions and categorical_crossentropy as loss functions*

# *3.Normalizing the x_train and x_test changing the pixel values from 0-255 into 0-1*

In [ ]:
x_train = normalize(x_train,axis=1)
x_test  = normalize(x_test,axis=1)

## *4. Spliting the training set into validation set*

In [ ]:
x_train_split, x_val, y_train_split, y_val = train_test_split(
    x_train, y_train,
    test_size=0.2,
    random_state=42,
    stratify=np.argmax(y_train, axis=1)  # ensure all the catergories distributes the same when running the code each times.
)

## *5. Data Augmentation*

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='reflect'
)

train_generator = train_datagen.flow(x_train_split,y_train_split,batch_size=20)



In [ ]:
# Printing all the dataset shapes
print("Dataset Shapes".center(50, '-'))
print(f"{'Training set x shape:':<25} {x_train.shape}")
print(f"{'Validation set x shape:':<25} {x_val.shape}")
print(f"{'Testing set x shape:':<25} {x_test.shape}")
print(f"{'Training set y shape:':<25} {y_train.shape}")
print(f"{'Validation set y shape:':<25} {y_val.shape}")
print(f"{'Testing set y shape:':<25} {y_test.shape}")
print("-" * 50)

*Checking the values of y_train and x_train beforehand to check* 

In [ ]:
plt.imshow(x_train[100])

In [ ]:
y_train[100]

In [ ]:
x_train[100]

In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D,BatchNormalization,Dropout,GlobalAveragePooling2D 
from keras import backend as K

In [ ]:
IMAGE_HEIGHT= x_train.shape[1]
IMAGE_WIDTH = x_train.shape[2]
IMAGE_CHANNELS= x_train.shape[3]

## Step2: Building a simple CNN model first

*This is an initial version of CNN model that I built, it is used to gain a basic understanding of an overview framework for this Multi-classification task as an initial experiment.*

*Some very vital model parameters definition which help better understand the model*
- *Loss function (categorial crossentropy) : Quantifies the error between output of the algorithm and given target value, in order to find the golabl minimum value by the optimizer shown here as adam.*

- *evaluated metrics: accuracy defined by the corrected number rate.*

- *Optimizers update the model in response to the output of the loss function.*

- *activation functions: importing non-linear factor to the results*

In [ ]:
# The initial CNN model structures: 
# activation = 'relu'
# model = Sequential()
# model.add(Conv2D(32, 3, activation = activation, padding = 'same', input_shape = (IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS)))
# model.add(BatchNormalization())

# model.add(Conv2D(32, 3, activation = activation, padding = 'same', kernel_initializer = 'he_uniform'))
# model.add(BatchNormalization())
# model.add(MaxPooling2D())

# model.add(Conv2D(64, 3, activation = activation, padding = 'same', kernel_initializer = 'he_uniform'))
# model.add(BatchNormalization())

# model.add(Conv2D(64, 3, activation = activation, padding = 'same', kernel_initializer = 'he_uniform'))
# model.add(BatchNormalization()) 
# model.add(MaxPooling2D())

# model.add(Flatten())
# model.add(Dense(256, activation = activation, kernel_initializer = 'he_uniform'))
# model.add(Dense(100, activation = 'softmax'))

# model.compile(optimizer = 'adam',loss = 'categorical_crossentropy', metrics = ['accuracy'])
# print(model.summary()) 

In [ ]:
# callbacks = [
#     tf.keras.callbacks.ModelCheckpoint(
#         filepath=r"C:\vscodeproject\Python\EE992\model_weight\model1_augmented_weight.h5", 
#         save_best_only=True, 
#         verbose=1,
#         monitor='val_loss', 
#         mode='min'           
#     )]

 
# history1 = model.fit_generator(
#     train_generator,
#     steps_per_epoch=1000,
#     epochs=25,
#     validation_data=(x_val, y_val),
#     callbacks=callbacks,  
#     verbose=1,
#     shuffle=False
# )


## Building more complex CNN model

In [ ]:
activation = 'relu'
model2 = Sequential()
model2.add(Conv2D(32, 3, activation = activation, padding = 'same', input_shape = (IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS)))
model2.add(BatchNormalization())

model2.add(Conv2D(32, 3, activation = activation, padding = 'same', kernel_initializer = 'he_uniform'))
model2.add(BatchNormalization())
model2.add(MaxPooling2D())

model2.add(Conv2D(64, 3, activation = activation, padding = 'same', kernel_initializer = 'he_uniform'))
model2.add(BatchNormalization())

model2.add(Conv2D(64, 3, activation = activation, padding = 'same', kernel_initializer = 'he_uniform'))
model2.add(BatchNormalization()) 
model2.add(MaxPooling2D())

model2.add(Conv2D(128, 3, activation = activation, padding = 'same', kernel_initializer = 'he_uniform'))
model2.add(BatchNormalization())

model2.add(Conv2D(128, 3, activation = activation, padding = 'same', kernel_initializer = 'he_uniform'))
model2.add(BatchNormalization()) 
model2.add(MaxPooling2D())

model2.add(Conv2D(256, 3, activation=activation, padding='same', kernel_initializer='he_uniform'))
model2.add(BatchNormalization())

model2.add(Conv2D(256, 3, activation=activation, padding='same', kernel_initializer='he_uniform'))
model2.add(BatchNormalization())
model2.add(MaxPooling2D())

model2.add(Flatten())
model2.add(Dense(1024, activation = activation, kernel_initializer = 'he_uniform'))
model2.add(Dropout(0.2))
model2.add(Dense(100, activation = 'softmax'))

model2.compile(optimizer = 'adam',loss = 'categorical_crossentropy', metrics = ['accuracy'])
print(model2.summary()) 

In [ ]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath=r"C:\vscodeproject\Python\EE992\model_weight\model2_augmented_best_val_acc.h5", 
        save_best_only=True, 
        verbose=1,
        monitor='val_accuracy', 
        mode='max'           
    )]

 
history2 = model2.fit_generator(
    train_generator,
    steps_per_epoch=1000,
    epochs=50,
    validation_data=(x_val, y_val),
    callbacks=callbacks,  
    verbose=1,
    shuffle=False
)


## Transfer learning

In [ ]:
# from tensorflow.keras.applications import ResNet50
# from tensorflow.keras.models import Model

# base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# x = Dense(256, activation='relu')(x)
# x = Dropout(0.2)(x)
# predictions = Dense(100, activation='softmax')(x)
# model3 = Model(inputs=base_model.input, outputs=predictions)

# for layer in base_model.layers:
#     layer.trainable = False

# model3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# print(model3.summary())

In [ ]:
# callbacks = [
#     tf.keras.callbacks.ModelCheckpoint(
#         filepath=r"C:\vscodeproject\Python\EE992\model_weight\model3_augmented_best_val_loss_transfer_learning.h5", 
#         save_best_only=True, 
#         verbose=1,
#         monitor='val_loss', 
#         mode='min'           
#     )]

 
# history3 = model3.fit_generator(
#     train_generator,
#     steps_per_epoch=1000,
#     epochs=50,
#     validation_data=(x_val, y_val),
#     callbacks=callbacks,  
#     verbose=1,
#     shuffle=False
# )


## Step3 : Visualisation of the validation and training sets of the loss and accuracy

- *plot the training and validation accuracy and loss at each epoch*

In [ ]:
loss = history2.history['loss']
val_loss = history2.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


*(These are just the logbook contents record my progressed processing based on the plot. Ticket means what I have done)*

- *Adding more layers*

- *Regularisation(dropout etc)*

- *Data augmentation ✔*

In [ ]:

acc = history2.history['accuracy']
val_acc = history2.history['val_accuracy']

plt.plot(epochs, acc, 'y', label='Training accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

## Step 4: Evaluate the training model on testing set

In [ ]:
y_pred = model2.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1) # Using argmax can return the index of the one-hot vector so that it can tell the certain type of the prediction class
y_true = np.argmax(y_test, axis=1)

In [ ]:
print(f'The shape of y_pred is:', y_pred_classes.shape)
print(f'The shape of y_true is:', y_true.shape)

In [ ]:
y_true

In [ ]:
# Printing the testing sample
num_samples = 10 # It is adjustable, can be changed to different number from 0-10000.
for i in range(num_samples):
    true_label = fine_label_names[y_true[i]]  # The true label name
    pred_label = fine_label_names[y_pred_classes[i]]  # The predicted type name
    print(f"Sample {i+1}:")
    print(f"  True Label: {true_label}")
    print(f"  Predicted Label: {pred_label}")
    print(f"  Correct: {true_label == pred_label}\n")

- *Visualization the images and categories*

In [ ]:
# Visualisation of the images and categories for the first 10 images
plt.figure(figsize=(15, 5))
for i in range(num_samples):
    plt.subplot(2, 5, i+1)
    plt.imshow(x_test[i])  # Showing the x_test data images
    true_label = fine_label_names[y_true[i]] # showing the true label catergories
    pred_label = fine_label_names[y_pred_classes[i]] # showing the prediction catergories
    plt.title(f"True: {true_label}\nPred: {pred_label}", fontsize=10)
    plt.axis('off')
plt.tight_layout()
plt.show()

Script(record the operations that I have tried):
- 1. After adding data augmentation the correct item improved from 1 to 2.
- 2. After optimizing the data augmentation hyperparameters such as the rotation_range from 45% to 15%, the correct itemes improved from 2 to 4.

## *Step5: Confusion matrix visualisation and classification report*

In [ ]:
# Compute the confusion matrix
cm = confusion_matrix(y_true, y_pred_classes)

# Find the top 10 catergories for better visualization the 100 X 100 confusion matrix otherwise it will be too dense to see the matrix.
top_k = 10
# Calculate the total number of predictions for each category (off-diagonal sum)
pred_counts = cm.sum(axis=0)  #  The sum of each column, indicating the total number projected for the category
top_k_indices = np.argsort(pred_counts)[-top_k:]  # Index of the top 10 most predicted categories

# Extracting the sub confusion matrix
sub_cm = cm[np.ix_(top_k_indices, top_k_indices)]
sub_label_names = [label_names[i] for i in top_k_indices]

# Visualiza the sub confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(sub_cm, annot=True, fmt='d', cmap='Blues', xticklabels=sub_label_names, yticklabels=sub_label_names)
plt.title('Confusion Matrix (Top 10 Predicted Classes)')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.show()

In [ ]:
# The classification report
report = classification_report(y_true, y_pred_classes, target_names=label_names)
print(report)

In [ ]:
test_loss, test_accuracy = model2.evaluate(x_test, y_test, verbose=1)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")